In [1]:
import pandas as pd
import glob
import os
import ner_functions as nf

In [2]:
# 定义一个字典储存每个国家的别名
country_aliases = {
"United States": ["USA", "America", "US", "United States", "UnitedStates"],
"Canada": ["Canada", "CA"],
"United Kingdom": ["UK", "United Kingdom", "Britain", "England", "Scotland", "Wales", "Northern Ireland", "UnitedKingdom"],
"Australia": ["Australia", "AU", "Aussie"],
"China": ["China", "PRC"],
"Denmark": ["Denmark", "DK"],
"Finland": ["Finland", "FI"],
"France": ["France", "French Republic", "FR"],
"Germany": ["Germany", "DE"],
"Japan": ["Japan", "JP"],
"Italy": ["Italy", "Italian Republic", "IT"],
"Netherlands": ["Netherlands", "Holland", "NL"],
"Norway": ["Norway", "NO"],
"Portugal": ["Portugal", "PT"],
"Singapore": ["Singapore", "SG"],
"South Korea": ["South Korea", "KR", "SouthKorea"],
"Spain": ["Spain", "ES"],
"Sweden": ["Sweden", "SE"],
"Switzerland": ["Switzerland", "Swiss Confederation", "Swiss", "CH"],
"New Zealand": ["New Zealand", "NZ", "NewZealand"]
}

In [3]:
lst_files = []

Path = "../Data/Articles/MWArticles/*.csv"

for fname in glob.glob(Path):
    lst_files.append(fname)

lst_files = sorted(lst_files)

# NER判定的阈值
threshold = 0

for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    country_name = country_name.replace("_articles", "")  # Remove the "_articles" part of the file name


    # 仅对单个国家的数据进行实体识别
    # if country_name == "China":
    # 对所有的未进行过NER的国家进行实体识别
    if os.path.isfile('../Data/count_3/MW_NER/{0}.csv'.format(country_name)):
        print("File already exists: {0}.csv".format(country_name))
    else:
        print(file)
        df = pd.read_csv(file)

        if df.shape[0] != 0:
            df = nf.preprocess_dataframe(df, use_parse=True)

            df, df_ner_results = nf.perform_ner_on_dataframe(df, country_name, country_aliases, threshold)

            # Convert the list of relevant articles back to a DataFrame
            df.to_csv(r'../Data/NER/count_3/MW_NER/{0}.csv'.format(country_name))
            df_ner_results.to_csv(r'../Data/NER/count_3/MW_NER_Results/{0}.csv'.format(country_name))

../Data/Articles/MWArticles\Australia_articles.csv


Processing Australia:   0%|          | 0/11105 [00:00<?, ?it/s]

../Data/Articles/MWArticles\Canada_articles.csv


Processing Canada:   0%|          | 0/12215 [00:00<?, ?it/s]

../Data/Articles/MWArticles\China_articles.csv


...

Processing Switzerland:   0%|          | 0/3092 [00:00<?, ?it/s]

../Data/Articles/MWArticles\United Kingdom_articles.csv


Processing United Kingdom:   0%|          | 0/2939 [00:00<?, ?it/s]

../Data/Articles/MWArticles\United States_articles.csv


Processing United States:   0%|          | 0/21938 [00:00<?, ?it/s]

In [5]:
# 读取所有国家的实体识别后剩余的数据, 然后输出剩余数据的数量
lst_ner_files = []
for fname in glob.glob("../Data/NER/count_3/MW_NER/*.csv"):
    lst_ner_files.append(fname)
for file in lst_ner_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    df = pd.read_csv(file)
    print(country_name, df.shape[0])

Australia 2108
Canada 1111
China 5317
Denmark 12
Finland 15
France 786
Germany 118
Italy 101
Japan 2751
Netherlands 53
New Zealand 467
Norway 51
Portugal 60
Singapore 401
South Korea 14
Spain 107
Sweden 68
Switzerland 200
United Kingdom 122
United States 1508


In [4]:
def process_single_article(text, country_name):
    ner_results = nlp(text)
    country_instances = [d for d in ner_results if (d['entity'] in "B-ORG") and (d['word'] in country_name) and (d['score'] > 0.98)]
    return len(country_instances)

../Data/countries_integration/Denmark_articles.csv


Processing Denmark:   0%|          | 0/1412 [00:00<?, ?it/s]

In [6]:
# Find all csv files in the path
csv_files = glob.glob('../Data/countries_integration/*.csv')

# Sort the file names
csv_files = sorted(csv_files)

# Read the first csv file
df = pd.read_csv(csv_files[0])

# Get the first row of the DataFrame
first_row = df.iloc[43]

# Extract the 'Date', 'Headline' and 'Text' columns
date = first_row['Date']
headline = first_row['Headline']
text = first_row['Text']

1412
839


In [10]:
ner_results

[{'entity': 'B-ORG',
  'score': 0.9932053,
  'index': 1,
  'word': 'AN',
  'start': 0,
  'end': 2},
 {'entity': 'I-ORG',
  'score': 0.992149,
  'index': 2,
  'word': '##Z',
  'start': 2,
  'end': 3},
 {'entity': 'B-ORG',
  'score': 0.99892753,
  'index': 4,
  'word': 'RB',
  'start': 9,
  'end': 11},
 {'entity': 'I-ORG',
  'score': 0.9988914,
  'index': 5,
  'word': '##A',
  'start': 11,
  'end': 12},
 {'entity': 'B-ORG',
  'score': 0.9988292,
  'index': 20,
  'word': 'Australia',
  'start': 56,
  'end': 65},
 {'entity': 'I-ORG',
  'score': 0.99927264,
  'index': 21,
  'word': '&',
  'start': 66,
  'end': 67},
 {'entity': 'I-ORG',
  'score': 0.9993352,
  'index': 22,
  'word': 'New',
  'start': 68,
  'end': 71},
 {'entity': 'I-ORG',
  'score': 0.9992262,
  'index': 23,
  'word': 'Zealand',
  'start': 72,
  'end': 79},
 {'entity': 'I-ORG',
  'score': 0.999316,
  'index': 24,
  'word': 'Banking',
  'start': 80,
  'end': 87},
 {'entity': 'I-ORG',
  'score': 0.99929905,
  'index': 25,
  'w